In [0]:
pip install nltk

Python interpreter will be restarted.
  Using cached nltk-3.7-py3-none-any.whl (1.5 MB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached click-8.0.4-py3-none-any.whl (97 kB)
  Using cached regex-2022.1.18-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (764 kB)
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-87292819-7dad-4fba-af40-2f204c121022/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.


In [0]:
book = sc.textFile("/FileStore/tables/sherlock.txt")

In [0]:
book.take(10)

Out[2]: ['The Project Gutenberg eBook of The Adventures of Sherlock Holmes, by Arthur Conan Doyle',
 '',
 'This eBook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this eBook or online at',
 'www.gutenberg.org. If you are not located in the United States, you',
 'will have to check the laws of the country where you are located before',
 'using this eBook.',
 '']

In [0]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
Out[3]: True

In [0]:
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords

In [0]:
# named entity recognizer
def ner(text):
    words = nltk.word_tokenize(text)
    posTaggedWords = nltk.pos_tag(words)
    wordChunks = nltk.ne_chunk(posTaggedWords)
    ans = []
    for chunk in wordChunks:
        if hasattr(chunk, 'label'):
            ans.append(' '.join(c[0] for c in chunk))
    return ans

In [0]:
text = book.reduce(lambda line1, line2 : line1 + " " + line2)
namedEntities = ner(text)
print(namedEntities)
print(len(namedEntities))

['Project Gutenberg', 'Adventures', 'Sherlock Holmes', 'Arthur Conan Doyle', 'United States', 'Project Gutenberg License', 'United States', 'Adventures of Sherlock Holmes', 'Arthur Conan Doyle Release Date', 'English Character', 'Jose Menendez', 'THE', 'PROJECT', 'ADVENTURES OF', 'SHERLOCK', 'Adventures of Sherlock Holmes', 'Arthur Conan Doyle', 'Bohemia', 'League', 'Identity', 'Boscombe Valley Mystery', 'Five Orange Pips', 'Twisted Lip', 'Adventure', 'Blue Carbuncle', 'Adventure', 'Speckled Band', 'Adventure', 'Engineer', 'Thumb X', 'Adventure', 'Noble Bachelor', 'Adventure', 'Beryl Coronet', 'Adventure', 'Copper', 'SCANDAL', 'Sherlock', 'Irene Adler', 'Irene Adler', 'Holmes', 'Holmes', 'Bohemian', 'Baker Street', 'Odessa', 'Trepoff', 'Atkinson', 'Trincomalee', 'Holland', 'Baker Street', 'Scarlet', 'Holmes', 'Watson', 'Seven', 'Watson', 'Mary Jane', 'Hence', 'London', 'Europe', 'Holmes', 'English', 'Holmes', 'Gesellschaft', 'German', 'Continental', 'Eglow', 'Egria', 'Bohemia', 'Carlsb

In [0]:
entityWords = sc.parallelize(namedEntities)

In [0]:
mappedWords = entityWords.map(lambda word : (word, 1))
#we use reduceByKey instead of reduce because reduce applies to each entity of data list but reduceByKey applies to pair key/values
wordCounts = mappedWords.reduceByKey(lambda x, y : x+y).sortBy(lambda x : -x[1])
wordCounts.collect()

Out[8]: [('Holmes', 273),
 ('Watson', 81),
 ('Sherlock Holmes', 73),
 ('Mr. Holmes', 68),
 ('Lestrade', 33),
 ('London', 32),
 ('Project', 29),
 ('Lord St. Simon', 27),
 ('Baker Street', 25),
 ('Doctor', 24),
 ('Project Gutenberg', 21),
 ('McCarthy', 20),
 ('Arthur', 20),
 ('Frank', 19),
 ('Mr. Rucastle', 19),
 ('England', 18),
 ('United States', 16),
 ('Miss Hunter', 16),
 ('City', 15),
 ('Miss Stoner', 14),
 ('Toller', 14),
 ('Irene Adler', 13),
 ('THE', 13),
 ('Mr. Windibank', 13),
 ('Mr. Wilson', 12),
 ('Mr. Holder', 12),
 ('German', 12),
 ('Roylott', 12),
 ('Mr. Merryweather', 11),
 ('Pray', 11),
 ('Foundation', 11),
 ('Peterson', 11),
 ('Mr. Sherlock Holmes', 10),
 ('Mr.', 10),
 ('America', 10),
 ('American', 10),
 ('Horner', 10),
 ('Boscombe Pool', 10),
 ('Horsham', 9),
 ('Bohemia', 9),
 ('Miss', 8),
 ('No', 8),
 ('Jones', 8),
 ('James', 8),
 ('Briony Lodge', 8),
 ('Stoke Moran', 8),
 ('Coroner', 8),
 ('Mr. Neville St. Clair', 8),
 ('Scotland Yard', 8),
 ('Countess', 8),
 ('Leag